In [1]:
import os
import json
import numpy as np
from spinn.data.nli.load_nli_data import convert_binary_bracketing
from spinn.util.catalan import ShiftProbabilities

In [2]:
# Sample Data

sample = [
    {"annotator_labels": ["neutral"], "genre": "government", "gold_label": "neutral", "pairID": "335730n", "promptID": "335730n", "sentence1": "Conceptually cream skimming has two basic dimensions - product and geography.", "sentence1_binary_parse": "( ( Conceptually ( cream skimming ) ) ( ( has ( ( ( two ( basic dimensions ) ) - ) ( ( product and ) geography ) ) ) . ) )", "sentence1_parse": "(ROOT (S (NP (JJ Conceptually) (NN cream) (NN skimming)) (VP (VBZ has) (NP (NP (CD two) (JJ basic) (NNS dimensions)) (: -) (NP (NN product) (CC and) (NN geography)))) (. .)))", "sentence2": "Product and geography are what make cream skimming work. ", "sentence2_binary_parse": "( ( ( Product and ) geography ) ( ( are ( what ( make ( cream ( skimming work ) ) ) ) ) . ) )", "sentence2_parse": "(ROOT (S (NP (NN Product) (CC and) (NN geography)) (VP (VBP are) (SBAR (WHNP (WP what)) (S (VP (VBP make) (NP (NP (NN cream)) (VP (VBG skimming) (NP (NN work)))))))) (. .)))"},
    {"annotator_labels": ["entailment"], "genre": "telephone", "gold_label": "entailment", "pairID": "249628e", "promptID": "249628e", "sentence1": "you know during the season and i guess at at your level uh you lose them to the next level if if they decide to recall the the parent team the Braves decide to call to recall a guy from triple A then a double A guy goes up to replace him and a single A guy goes up to replace him", "sentence1_binary_parse": "( you ( ( know ( during ( ( ( the season ) and ) ( i guess ) ) ) ) ( at ( at ( ( your level ) ( uh ( you ( ( ( lose them ) ( to ( the ( next level ) ) ) ) ( if ( ( if ( they ( decide ( to ( recall ( the ( the ( parent team ) ) ) ) ) ) ) ) ( ( the Braves ) ( decide ( to ( call ( to ( ( recall ( a guy ) ) ( from ( ( triple A ) ( ( ( then ( ( a ( double ( A guy ) ) ) ( ( goes up ) ( to ( replace him ) ) ) ) ) and ) ( ( a ( single ( A guy ) ) ) ( ( goes up ) ( to ( replace him ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) )", "sentence1_parse": "(ROOT (S (NP (PRP you)) (VP (VBP know) (PP (IN during) (NP (NP (DT the) (NN season)) (CC and) (NP (FW i) (FW guess)))) (PP (IN at) (IN at) (NP (NP (PRP$ your) (NN level)) (SBAR (S (INTJ (UH uh)) (NP (PRP you)) (VP (VBP lose) (NP (PRP them)) (PP (TO to) (NP (DT the) (JJ next) (NN level))) (SBAR (IN if) (S (SBAR (IN if) (S (NP (PRP they)) (VP (VBP decide) (S (VP (TO to) (VP (VB recall) (NP (DT the) (DT the) (NN parent) (NN team)))))))) (NP (DT the) (NNPS Braves)) (VP (VBP decide) (S (VP (TO to) (VP (VB call) (S (VP (TO to) (VP (VB recall) (NP (DT a) (NN guy)) (PP (IN from) (NP (NP (RB triple) (DT A)) (SBAR (S (S (ADVP (RB then)) (NP (DT a) (JJ double) (NNP A) (NN guy)) (VP (VBZ goes) (PRT (RP up)) (S (VP (TO to) (VP (VB replace) (NP (PRP him))))))) (CC and) (S (NP (DT a) (JJ single) (NNP A) (NN guy)) (VP (VBZ goes) (PRT (RP up)) (S (VP (TO to) (VP (VB replace) (NP (PRP him))))))))))))))))))))))))))))", "sentence2": "You lose the things to the following level if the people recall.", "sentence2_binary_parse": "( You ( ( ( ( lose ( the things ) ) ( to ( the ( following level ) ) ) ) ( if ( ( the people ) recall ) ) ) . ) )", "sentence2_parse": "(ROOT (S (NP (PRP You)) (VP (VBP lose) (NP (DT the) (NNS things)) (PP (TO to) (NP (DT the) (JJ following) (NN level))) (SBAR (IN if) (S (NP (DT the) (NNS people)) (VP (VBP recall))))) (. .)))"},
    {"annotator_labels": ["entailment"], "genre": "fiction", "gold_label": "entailment", "pairID": "169837e", "promptID": "169837e", "sentence1": "One of our number will carry out your instructions minutely.", "sentence1_binary_parse": "( ( One ( of ( our number ) ) ) ( ( will ( ( ( carry out ) ( your instructions ) ) minutely ) ) . ) )", "sentence1_parse": "(ROOT (S (NP (NP (CD One)) (PP (IN of) (NP (PRP$ our) (NN number)))) (VP (MD will) (VP (VB carry) (PRT (RP out)) (NP (PRP$ your) (NNS instructions)) (ADVP (RB minutely)))) (. .)))", "sentence2": "A member of my team will execute your orders with immense precision.", "sentence2_binary_parse": "( ( ( A member ) ( of ( my team ) ) ) ( ( will ( ( execute ( your orders ) ) ( with ( immense precision ) ) ) ) . ) )", "sentence2_parse": "(ROOT (S (NP (NP (DT A) (NN member)) (PP (IN of) (NP (PRP$ my) (NN team)))) (VP (MD will) (VP (VB execute) (NP (PRP$ your) (NNS orders)) (PP (IN with) (NP (JJ immense) (NN precision))))) (. .)))"},
]

In [3]:
# Globals

shift_probabilites = ShiftProbabilities()

In [4]:
# Helper Methods

def product(p):
    return reduce(lambda x, y: x*y, p, 1.0)

def binary_parse(sentence, transitions):
    """
    Generate a binary parse string using shift-reduce transitions.
    """
    buf = list(reversed(sentence))
    stack = []
    
    for t in transitions:
        if t == 0:
            stack.append(buf.pop())
        elif t == 1:
            right = stack.pop()
            left = stack.pop()
            stack.append('( ' + left + ' ' + right + ' )')

    return stack[0]

def uniform_random_transitions(N):
    """
    Given a desired sentence length, generate a sequence of shift-reduce
    transitions by uniformly sampling a transition at each timestep.
    """
    seq_length = N*2-1
    buf = range(N)
    stack = []
    ts = []

    for i in range(seq_length):
        t = np.random.choice([0, 1])
        if len(stack) < 2: # Can't reduce
            t = 0
        if len(buf) < 1: # Can't shift
            t = 1
        if t == 0:
            stack.append(buf.pop())
        if t == 1:
            stack.append(stack.pop() + stack.pop())
        ts.append(t)

    return ts

def uniform_random_trees(N):
    """
    Given a desired sentence length, generate a sequence of shift-reduce
    transitions such that the sequence is uniformly random. At each
    timestep sample from the catalan pyramid distribution.
    """
    seq_length = N*2-1
    buf = range(N)
    stack = []
    ts = []
    
    N_reduces = 0
    
    for i in range(seq_length):
        shift_probability = shift_probabilites.prob(N_reduces, i, N)
        p = [shift_probability, 1. - shift_probability]
        t = np.random.choice([0, 1], p=p)
        if len(stack) < 2: # Can't reduce
            t = 0
        if len(buf) < 1: # Can't shift
            t = 1
        if t == 0:
            stack.append(buf.pop())
        if t == 1:
            N_reduces += 1
            stack.append(stack.pop() + stack.pop())
        ts.append(t)

    return ts

def brute_force_transitions_helper(N_stack, N_buf, N_remaining):
    """
    Return all possible suffixes.
    """
    suffixes = []
    probabilities = []
    if N_remaining == 1:
        suffixes.append([1])
        probabilities.append([1.0])
    else:
        cant_shift = N_buf == 0
        cant_reduce = N_stack < 2
        shift_probability = 1.0 if cant_reduce else 0.5
        reduce_probability = 1.0 if cant_shift else 0.5
        if not cant_shift:
            s, p = brute_force_transitions_helper(N_stack + 1, N_buf - 1, N_remaining - 1)
            for suffix, probability in zip(s, p):
                suffixes.append([0] + suffix)
                probabilities.append([shift_probability] + probability)
        if not cant_reduce:
            s, p = brute_force_transitions_helper(N_stack - 1, N_buf, N_remaining - 1)
            for suffix, probability in zip(s, p):
                suffixes.append([1] + suffix)
                probabilities.append([reduce_probability] + probability)
    return suffixes, probabilities

def brute_force_transitions(N):
    """
    Given a desired length, generate all possible sequences of shift-reduce
    transitions. Also return the expectation of each sequence.
    """
    seq_length = N*2-1
    
    sequences, probabilities = brute_force_transitions_helper(N_stack=0, N_buf=N, N_remaining=seq_length)
    
    return sequences, probabilities

def catalan_transitions_helper(N_reduces, timestep, N):
    """
    Return all possible suffixes.
    """
    suffixes = []
    probabilities = []
    if timestep == 2 * N - 2:
        suffixes.append([1])
        probabilities.append([1.0])
    else:
        shift_probability = shift_probabilites.prob(N_reduces, timestep, N)
        reduce_probability = 1.0 - shift_probability
        if shift_probability > 0:
            s, p = catalan_transitions_helper(N_reduces, timestep + 1, N)
            for suffix, probability in zip(s, p):
                suffixes.append([0] + suffix)
                probabilities.append([shift_probability] + probability)
        if reduce_probability > 0:
            s, p = catalan_transitions_helper(N_reduces + 1, timestep + 1, N)
            for suffix, probability in zip(s, p):
                suffixes.append([1] + suffix)
                probabilities.append([reduce_probability] + probability)
    return suffixes, probabilities

def catalan_transitions(N):
    """
    Given a desired length, generate all possible sequences of shift-reduce
    transitions. Also return the expectation of each sequence.
    """
    seq_length = N*2-1
    
    sequences, probabilities = catalan_transitions_helper(N_reduces=0, timestep=0, N=N)
    
    return sequences, probabilities

# Demos

In [14]:
# Check: Successfully generate a binary parse string.

parse = sample[0]['sentence1_binary_parse']
sentence, transitions = convert_binary_bracketing(parse)

actual = parse
expected = binary_parse(sentence, transitions)

assert actual == expected, 'Did not parse correctly.'

# repeat for posterity
expected = binary_parse(sentence, transitions)

assert actual == expected, 'Running binary_parse twice is problematic.'

# negative test
expected = binary_parse(sentence, transitions)

assert 'actual' != expected

In [6]:
# Demo: Generate a handful of random trees for a fixed length.

rounds = 10
N = 10 # sentence length

for _ in range(rounds):
    tree = uniform_random_transitions(N)
    print(tree, len(tree))

([0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1], 19)
([0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1], 19)
([0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1], 19)
([0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1], 19)
([0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1], 19)
([0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1], 19)
([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 19)
([0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1], 19)
([0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1], 19)
([0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1], 19)


In [7]:
# Demo: All possible sequences of specified length.

N = 5

def product(p):
    return reduce(lambda x, y: x*y, p, 1.0)

sequences, probabilities = brute_force_transitions(N)
for s, p in zip(sequences, probabilities):
    print(s, p, product(p))
    
print(len(sequences))
    
print(sum(map(lambda x: product(x), probabilities)))

([0, 0, 0, 0, 0, 1, 1, 1, 1], [1.0, 1.0, 0.5, 0.5, 0.5, 1.0, 1.0, 1.0, 1.0], 0.125)
([0, 0, 0, 0, 1, 0, 1, 1, 1], [1.0, 1.0, 0.5, 0.5, 0.5, 0.5, 1.0, 1.0, 1.0], 0.0625)
([0, 0, 0, 0, 1, 1, 0, 1, 1], [1.0, 1.0, 0.5, 0.5, 0.5, 0.5, 0.5, 1.0, 1.0], 0.03125)
([0, 0, 0, 0, 1, 1, 1, 0, 1], [1.0, 1.0, 0.5, 0.5, 0.5, 0.5, 0.5, 1.0, 1.0], 0.03125)
([0, 0, 0, 1, 0, 0, 1, 1, 1], [1.0, 1.0, 0.5, 0.5, 0.5, 0.5, 1.0, 1.0, 1.0], 0.0625)
([0, 0, 0, 1, 0, 1, 0, 1, 1], [1.0, 1.0, 0.5, 0.5, 0.5, 0.5, 0.5, 1.0, 1.0], 0.03125)
([0, 0, 0, 1, 0, 1, 1, 0, 1], [1.0, 1.0, 0.5, 0.5, 0.5, 0.5, 0.5, 1.0, 1.0], 0.03125)
([0, 0, 0, 1, 1, 0, 0, 1, 1], [1.0, 1.0, 0.5, 0.5, 0.5, 1.0, 0.5, 1.0, 1.0], 0.0625)
([0, 0, 0, 1, 1, 0, 1, 0, 1], [1.0, 1.0, 0.5, 0.5, 0.5, 1.0, 0.5, 1.0, 1.0], 0.0625)
([0, 0, 1, 0, 0, 0, 1, 1, 1], [1.0, 1.0, 0.5, 1.0, 0.5, 0.5, 1.0, 1.0, 1.0], 0.125)
([0, 0, 1, 0, 0, 1, 0, 1, 1], [1.0, 1.0, 0.5, 1.0, 0.5, 0.5, 0.5, 1.0, 1.0], 0.0625)
([0, 0, 1, 0, 0, 1, 1, 0, 1], [1.0, 1.0, 0.5, 1.0, 0.5, 0.5, 0.

In [8]:
# Demo: All possible sequences of specified length.

N = 5

sequences, probabilities = catalan_transitions(N)
for s, p in zip(sequences, probabilities):
    print(s, list(map(lambda x: round(x, 3), p)), product(p))
    
print(len(sequences))
    
print(sum(map(lambda x: product(x), probabilities)))

([0, 0, 0, 0, 0, 1, 1, 1, 1], [1.0, 1.0, 0.643, 0.444, 0.25, 1.0, 1.0, 1.0, 1.0], 0.07142857142857142)
([0, 0, 0, 0, 1, 0, 1, 1, 1], [1.0, 1.0, 0.643, 0.444, 0.75, 0.333, 1.0, 1.0, 1.0], 0.07142857142857142)
([0, 0, 0, 0, 1, 1, 0, 1, 1], [1.0, 1.0, 0.643, 0.444, 0.75, 0.667, 0.5, 1.0, 1.0], 0.07142857142857144)
([0, 0, 0, 0, 1, 1, 1, 0, 1], [1.0, 1.0, 0.643, 0.444, 0.75, 0.667, 0.5, 1.0, 1.0], 0.07142857142857144)
([0, 0, 0, 1, 0, 0, 1, 1, 1], [1.0, 1.0, 0.643, 0.556, 0.6, 0.333, 1.0, 1.0, 1.0], 0.07142857142857144)
([0, 0, 0, 1, 0, 1, 0, 1, 1], [1.0, 1.0, 0.643, 0.556, 0.6, 0.667, 0.5, 1.0, 1.0], 0.07142857142857145)
([0, 0, 0, 1, 0, 1, 1, 0, 1], [1.0, 1.0, 0.643, 0.556, 0.6, 0.667, 0.5, 1.0, 1.0], 0.07142857142857145)
([0, 0, 0, 1, 1, 0, 0, 1, 1], [1.0, 1.0, 0.643, 0.556, 0.4, 1.0, 0.5, 1.0, 1.0], 0.07142857142857144)
([0, 0, 0, 1, 1, 0, 1, 0, 1], [1.0, 1.0, 0.643, 0.556, 0.4, 1.0, 0.5, 1.0, 1.0], 0.07142857142857144)
([0, 0, 1, 0, 0, 0, 1, 1, 1], [1.0, 1.0, 0.357, 1.0, 0.6, 0.333, 1

In [9]:
# Demo: Generate a handful of random trees for a fixed length
# using the catalan pyramid distribution.

rounds = 10
N = 10 # sentence length

for _ in range(rounds):
    tree = uniform_random_trees(N)
    print(tree, len(tree))

([0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1], 19)
([0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1], 19)
([0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1], 19)
([0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1], 19)
([0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1], 19)
([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1], 19)
([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1], 19)
([0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1], 19)
([0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1], 19)
([0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1], 19)


# Main

In [10]:
# Source Data

data = [
    dict(source = '~/data/multinli_1.0/multinli_1.0_dev_mismatched.jsonl',
    target_uniform_transitions = '~/data/multinli_1.0/multinli_1.0_dev_mismatched-random_transitions.jsonl',
    target_uniform_trees = '~/data/multinli_1.0/multinli_1.0_dev_mismatched-random_trees.jsonl',
    ),
    dict(source = '~/data/multinli_1.0/multinli_1.0_dev_matched.jsonl',
    target_uniform_transitions = '~/data/multinli_1.0/multinli_1.0_dev_matched-random_transitions.jsonl',
    target_uniform_trees = '~/data/multinli_1.0/multinli_1.0_dev_matched-random_trees.jsonl',
    ),
    dict(source = '~/data/multinli_1.0/multinli_1.0_train.jsonl',
    target_uniform_transitions = '~/data/multinli_1.0/multinli_1.0_train-random_transitions.jsonl',
    target_uniform_trees = '~/data/multinli_1.0/multinli_1.0_train-random_trees.jsonl',
    )
]

In [11]:
# Helper Methods

def write_dataset(dataset, target):
    with open(target, 'w') as f:
        for example in dataset:
            f.write(json.dumps(example) + '\n')

def read_and_seperate_parses(source):
    sentence1_tokens = []
    sentence2_tokens = []
    dataset = []
    with open(source) as f:
        for line in f:
            try:
                line = line.encode('UTF-8')
            except UnicodeError as e:
                print "ENCODING ERROR:", line, e
                line = "{}"

            loaded_example = json.loads(line)

            tokens1, _ = convert_binary_bracketing(loaded_example["sentence1_binary_parse"])
            sentence1_tokens.append(tokens1)
            del loaded_example["sentence1_binary_parse"]

            tokens2, _ = convert_binary_bracketing(loaded_example["sentence2_binary_parse"])
            sentence2_tokens.append(tokens2)
            del loaded_example["sentence2_binary_parse"]
            
            dataset.append(loaded_example)
    return dataset, sentence1_tokens, sentence2_tokens

def generate_random_transitions(dataset, sentence1_tokens, sentence2_tokens):
    for example, s1, s2 in zip(dataset, sentence1_tokens, sentence2_tokens):
        s1_transitions = uniform_random_transitions(len(s1))
        s1_binary_parse = binary_parse(s1, s1_transitions)
        example['sentence1_binary_parse'] = s1_binary_parse
        
        s2_transitions = uniform_random_transitions(len(s2))
        s2_binary_parse = binary_parse(s2, s2_transitions)
        example['sentence2_binary_parse'] = s2_binary_parse
    return dataset

def generate_random_trees(dataset, sentence1_tokens, sentence2_tokens):
    for example, s1, s2 in zip(dataset, sentence1_tokens, sentence2_tokens):
        s1_transitions = uniform_random_trees(len(s1))
        s1_binary_parse = binary_parse(s1, s1_transitions)
        example['sentence1_binary_parse'] = s1_binary_parse
        
        s2_transitions = uniform_random_trees(len(s2))
        s2_binary_parse = binary_parse(s2, s2_transitions)
        example['sentence2_binary_parse'] = s2_binary_parse
    return dataset

def generate(source, target_uniform_transitions, target_uniform_trees):
    """
    1. Read dataset into memory.
    2. Create copy using uniform transitions.
    3. Create copy using uniform trees.
    """
    print("reading: {}".format(source))
    dataset, sentence1_tokens, sentence2_tokens = read_and_seperate_parses(source)
    
    assert len(sentence1_tokens) == len(sentence2_tokens)
    assert len(sentence1_tokens) == len(dataset)
    
    # Random Transitions
    print("building: {}".format(target_uniform_transitions))
    dataset_random_transitions = generate_random_transitions(dataset, sentence1_tokens, sentence2_tokens)
    print("writing: {}".format(target_uniform_transitions))
    write_dataset(dataset_random_transitions, target_uniform_transitions)
    
    # Random Trees
    print("building: {}".format(target_uniform_trees))
    dataset_random_trees = generate_random_trees(dataset, sentence1_tokens, sentence2_tokens)
    print("writing: {}".format(target_uniform_trees))
    write_dataset(dataset_random_trees, target_uniform_trees)

In [12]:
def run(data_dicts, rng_seed=11):
    np.random.seed(rng_seed)
    for dd in data_dicts:
        generate(
            os.path.expanduser(dd['source']), 
            os.path.expanduser(dd['target_uniform_transitions']),
            os.path.expanduser(dd['target_uniform_trees'])
        )
    
run(data)

reading: /home/dexter/data/multinli_1.0/multinli_1.0_dev_mismatched.jsonl
building: /home/dexter/data/multinli_1.0/multinli_1.0_dev_mismatched-random_transitions.jsonl
writing: /home/dexter/data/multinli_1.0/multinli_1.0_dev_mismatched-random_transitions.jsonl
building: /home/dexter/data/multinli_1.0/multinli_1.0_dev_mismatched-random_trees.jsonl
writing: /home/dexter/data/multinli_1.0/multinli_1.0_dev_mismatched-random_trees.jsonl
reading: /home/dexter/data/multinli_1.0/multinli_1.0_dev_matched.jsonl
building: /home/dexter/data/multinli_1.0/multinli_1.0_dev_matched-random_transitions.jsonl
writing: /home/dexter/data/multinli_1.0/multinli_1.0_dev_matched-random_transitions.jsonl
building: /home/dexter/data/multinli_1.0/multinli_1.0_dev_matched-random_trees.jsonl
writing: /home/dexter/data/multinli_1.0/multinli_1.0_dev_matched-random_trees.jsonl
reading: /home/dexter/data/multinli_1.0/multinli_1.0_train.jsonl
building: /home/dexter/data/multinli_1.0/multinli_1.0_train-random_transitions